##hash function

In [9]:
import math
import time
import itertools
start = time.time()

def hash(element1,element2,num_trans):
  result = (element1 + element2) % num_trans
  return result

##Load Data && generate C1 and pairs for bucket

In [10]:
candidates = {}
buckets = {}
num_trans = 0
with open('retail.txt', 'r') as file:
    for line in file:
      num_trans += 1

def read_data(num_trans,candidates,buckets):
  all_data = []
  with open('retail.txt', 'r') as file:
    for line in file:
      line_list = [int(x) for x in line.strip().split()]
      all_data.append(line_list)
      for item in line_list:
        if item in candidates.keys():
          candidates.update({item:(candidates[item]+1)})
        else:
          candidates.update({item:1})
      pairs = list(itertools.combinations(line_list, 2))
      for pair in pairs:
        index = hash(pair[0], pair[1],num_trans)
        if index in buckets.keys():
          buckets.update({index:(buckets[index]+1)})
        else:
          buckets.update({index:1})
  return all_data,candidates,buckets
#netflix.data retail.txt

# get transactions,c1,buckets
all_data,c1,buckets = read_data(num_trans,candidates,buckets)
min_support = 0.02
#0.01

##Generate Bitmap


In [11]:
def generate_Bitmap(hashtable,num_trans,min_support):
  bitmap = [0]*num_trans
  for bucket,bucket_count in hashtable.items():
    if bucket_count/num_trans >= min_support:
      bitmap[bucket] = 1
  return bitmap

##Generate L1

In [12]:
def get_L(candidates,num_trans,min_support):
  L = []
  for key,value in candidates.items():
    if value/num_trans >= min_support:
      L.append(key)
  return L

L1 = get_L(c1,num_trans,min_support)

##Prune Dataset

In [13]:
def prune_dataset(candidates,transactions):
  pruned_dataset = []
  for basket in transactions:
    for item in basket:
      if item in candidates:
        pruned_dataset.append(basket)
        break
  return pruned_dataset

pruned_dataset = prune_dataset(L1,all_data)

##Count initially_pruned C2

In [14]:
def get_C2(candidates,dataset):
  for basket in dataset:
    for key in candidates.keys():
      if set(key).issubset(set(basket)):
        candidates.update({key:candidates[key]+1})
  return candidates

##Generate C2 && bitmap

In [15]:
C2 = list(itertools.combinations(L1, 2))

bitmap = generate_Bitmap(buckets,num_trans,min_support)

# count = 0
# for pair in C2:
#   if hash(pair[0],pair[1]) in buckets.keys():
#     count += 1


##Generate L2

In [ ]:
def get_L2(C2,bitmap,num_trans,pruned_dataset,min_support):
  for pair in C2:
    index = hash(pair[0], pair[1],num_trans)
    if bitmap[index] != 1:
      C2.remove(pair)
      
  if len(C2) == 0:
    return None
  else:
    initial_value = [0]*len(C2)
    C2_initial = dict(zip(C2, initial_value))
    C2_initial = get_C2(C2_initial,pruned_dataset)
    L2 = get_L(C2_initial,num_trans,min_support)
    return L2

L2 = get_L2(C2,bitmap,num_trans,pruned_dataset,min_support)
print(len(L2))
for pair in L2:
  print(pair)
print("--- seconds---" , (time.time() - start))